## KNN
Also implemented in lab4, which has more parameters

In [6]:
import numpy as np

class Point:
    def __init__(self, name, x, y) -> None:
        self.name = name
        self.x = x
        self.y = y

    def distance(self, point):
        return (self.x-point.x)**2 + (self.y-point.y)**2


names = [chr(i) for i in range(65, 75)]
xs = [352, 933, 539, 293, 512, 23, 4, 1077, 1033, 701]
ys = [768, 1093, 192, 422, 858, 444, 9, 380, 905, 639]

points = [Point(name, x, y) for name, x, y in zip(names, xs, ys)]

# for i, p in enumerate(points):
#     for j, q in enumerate(points):
#         if j < i:
#             print("    ", end='\t')
#         else:
#             print(p.distance(q), end='\t')
#     print()
dists = [p.distance(q) for i, p in enumerate(points) for j, q in enumerate(points) if j > i]
print(min(dists))


33700


In [11]:
import numpy as np

class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y

    def distance(self, point):
        return (self.x-point.x)**2 + (self.y-point.y)**2
    
    def __str__(self) -> str:
        return f"({self.x}, {self.y})"

    def __repr__(self) -> str:
        return self.__str__()

# names = [chr(i) for i in range(65, 75)]
xs = [352, 933, 539, 293, 512, 23, 4, 1077, 1033, 701]
ys = [768, 1093, 192, 422, 858, 444, 9, 380, 905, 639]
points = [Point(x, y) for x, y in zip(xs, ys)]

centroids = [Point(400,10), Point(300,700), Point(800,300)]
classification = [[] for _ in range(3)]
while True:
    for point in points:
        dists = [point.distance(centroid) for centroid in centroids]
        cate = dists.index(min(dists))
        classification[cate].append(point)
    print(classification)
    break

[[(539, 192), (4, 9)], [(352, 768), (933, 1093), (293, 422), (512, 858), (23, 444)], [(1077, 380), (1033, 905), (701, 639)]]


## Hierarchical Clustering


In [4]:
data = """A B C D E F G H I J
x 352 933 539 293 512 23 4 1077 1033 701
y 768 1093 192 422 858 444 9 380 905 639"""

def convert_to_csv(data):
    names, xs, ys = data.split('\n')
    names = names.split()
    _, *xs = xs.split()
    _, *ys = ys.split()
    txt = ""
    for name, x, y in zip(names, xs, ys):
        txt += f"{x},{y},{name}\n"
    return txt

txt = convert_to_csv(data)
print(txt)

352,768,A
933,1093,B
539,192,C
293,422,D
512,858,E
23,444,F
4,9,G
1077,380,H
1033,905,I
701,639,J



In [41]:
class Cluster:
    def __init__(self, points):
        self.points = points
        self.name = ''.join(sorted([p.name for p in points]))
    
    def distance(self, cluster, linkage_type):
        dists = []
        for p1 in self.points:
            for p2 in cluster.points:
                dists.append(p1.distance(p2))
        return min(dists) if linkage_type == 'single' else max(dists)
    
    def __add__(self, cluster):
        return Cluster(self.points + cluster.points)



In [43]:
def parse_input(txt):
    point_lst = []
    for line in txt.split("\n"):
        if not line:
            continue
        x, y, name = line.split(',')
        point_lst.append(Point(name, int(x), int(y)))
    return point_lst

from math import inf

def hierarchical_clustering(points, k, linkage_type='single'):
    dist_table = {}
    clusters = {c.name: c for c in [Cluster([point]) for point in points]}
    while len(clusters) > k:
        distance_list = []
        for c1 in clusters:
            for c2 in clusters:
                if c1 < c2:
                    distance_list.append((c1, c2, clusters[c1].distance(clusters[c2], linkage_type)))
        c1, c2, _ = min(distance_list, key=lambda x: x[2])
        new_cluster = clusters[c1] + clusters[c2]
        clusters[new_cluster.name] = new_cluster
        clusters.pop(c1)
        clusters.pop(c2)
        print(' '.join(sorted(clusters.keys())))

points = parse_input(txt)
hierarchical_clustering(points, 3, 'single')
hierarchical_clustering(points, 3, 'complete')


AE B C D F G H I J
AE BI C D F G H J
AE BI C DF G H J
AEJ BI C DF G H
AEJ BI CDF G H
ACDEFJ BI G H
ABCDEFIJ G H
AE B C D F G H I J
AE BI C D F G H J
AE BI C DF G H J
AEJ BI C DF G H
AEJ BI C DFG H
AEJ BI CH DFG
ABEIJ CH DFG
